In [1]:
# -----------------------------------------------------------------------------
# Copyright 2020 by ShabaniPy Authors, see AUTHORS for more details.
#
# Distributed under the terms of the MIT license.
#
# The full license is in the file LICENCE, distributed with this software.
# -----------------------------------------------------------------------------
"""Example of using data classification tools for Shapiro experiments.

"""
#: List of path to inspect looking for raw data
FOLDERS = ["/Users/bh/Labber/Data/2020/12/Data_1202"]

#: Name of the sample that must appear in the measurement name.
SAMPLE_NAME = "2JJ-2HB-5MGJJ-MG2"

#: Path under which saving the consolidated data.
CONSOLIDATION_PATH = ("/Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/"
"JS311/MG2/General/JS311-MG2_raw_data-Gen-051.hdf5")

#: Path to a file in which to store the log of the aggregation
LOG_PATH = (
    "/Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/Log/"
    "JS311-MG2/JS311-MG2_aggregation.log"
)

import logging
import os

import numpy as np
import h5py

from shabanipy.bulk.data_classifying import (
    DataClassifier,
    FilenamePattern,
    LogPattern,
    MeasurementPattern,
    NamePattern,
    RampPattern,
    StepPattern,
    ValuePattern,
)

# Parametrize the logging system
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
# ch = logging.StreamHandler(None)
# ch.setLevel(logging.DEBUG)
# logger.addHandler(ch)
ch = logging.FileHandler(LOG_PATH)
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)


# --- Configuation ---------------------------------------------------------------------

# Get large bias measurements and Shapiro with a minimum number of points in power

clf = DataClassifier(
    patterns=[
        MeasurementPattern(
            name="IcRn",
            filename_pattern=FilenamePattern(
                regex=f"^JS311-.+(?P<sample>{SAMPLE_NAME})-051.+$",
                use_in_classification=True,
            ),
            steps=[
#                 StepPattern(
#                     name="Bias",
#                     #name_pattern=1,
#                     name_pattern=NamePattern(names=["VITracer - VI curve"]),
#                     #ramps=[RampPattern(span=ValuePattern(greater=0))],
#                 ),
                #"""Uncomment when running with data with forward and backward bias"""
#                 StepPattern(
#                     name="Counter",
#                     name_pattern=1,
#                     ramps=[RampPattern(points=ValuePattern(value=2))],
#                 ),
#                 StepPattern(
#                     name="In-plane Field",
#                     name_pattern=NamePattern(names=["Vector Magnet - Field Z"]),
#                     use_in_classification=True,
#                     classifier_level=1,
#                 ),
#                 StepPattern(
#                     name="Out of Plane Field",
#                     name_pattern=NamePattern(names=["Magnet Source - Source current"]),
# #                     use_in_classification=True,
# #                     classifier_level=1,
#                 ),
#                 StepPattern(
#                     name="CM: GV",
#                     name_pattern=NamePattern(names=["SM1 - Source voltage"]),
#                     use_in_classification=True,
#                     classifier_level=1,
#                 ),
                StepPattern(
                    name="GV",
                    name_pattern=NamePattern(names=["SM2 - Source voltage"]),
                    use_in_classification=True,
                    classifier_level=1,
                ),
            ],
            logs=[
#                 LogPattern(name="Voltage Drop", pattern=0),
                LogPattern(name="Voltage Drop", pattern=NamePattern(names=("VITracer - VI curve")),x_name = "Bias"),
            ],
        ),
#         MeasurementPattern(
#             name="Shapiro",
#             filename_pattern=FilenamePattern(
#                 regex=f"^JS124R.+-(?P<sample>{SAMPLE_NAME})-.+$",
#                 use_in_classification=True,
#             ),
#             steps=[
#                 StepPattern(name="Bias", name_pattern=0, ramps=[RampPattern()],),
#                 StepPattern(
#                     name="RF ON",
#                     name_pattern=NamePattern(names=["BNC - Output"]),
#                     value=ValuePattern(value=1),
#                 ),
#                 StepPattern(
#                     name="RF Power",
#                     name_pattern=NamePattern(names=["BNC - Power"]),
#                     ramps=[RampPattern(points=ValuePattern(greater=5))],
#                 ),
# #                 StepPattern(
# #                     name="Field",
# #                     name_pattern=NamePattern(names=["Vector Magnet - Field X"]),
# #                     use_in_classification=True,
# #                     classifier_level=1,
# #                 ),
#                 StepPattern(
#                     name="Frequency",
#                     name_pattern=NamePattern(names=["BNC - Frequency"]),
#                     use_in_classification=True,
#                     classifier_level=2,
#                 ),
#             ],
#             logs=[
#                 LogPattern(name="Voltage Drop", pattern=NamePattern(names=("DMM - Value",))),
#                 #LogPattern(name="JJ Current", pattern=NamePattern(names=("DMM2 - Value",))),
#             ],
#         ),
    ]
)

# --- Run ------------------------------------------------------------------------------

clf.identify_datasets(FOLDERS)

for n, datasets in clf._datasets.items():
    print(f"Measurement: {n}")
    for d in datasets:
        print(f"  - {d.rsplit(os.sep, 1)[-1]}")

clf.classify_datasets()
for n, ds in clf._classified_datasets.items():
    print(f"Measurement: {n}")
    for path, classifiers in ds.items():
        print(f"  - {d.rsplit(os.sep, 1)[-1]}")
        for level, values in classifiers.items():
            print(f"    - level {level}: {values}")

clf.consolidate_dataset(CONSOLIDATION_PATH)


def pretty_print(root: h5py.Group, increment=""):
    for k, v in root.items():
        if isinstance(v, h5py.Group):
            print(increment + f"- Group {k}:")
            print(increment + f"  {dict(v.attrs)}")
            pretty_print(v, increment + "  ")
        elif isinstance(v, h5py.Dataset):
            print(
                increment + f"- Dataset {k}: shape {v.shape}, dtype {v.dtype}, "
                f"min {np.min(v[...])}, max {np.min(v[...])}"
            )


with h5py.File(CONSOLIDATION_PATH,'r') as f:
    pretty_print(f)



Measurement: IcRn
  - JS311-BHENL001-2JJ-2HB-5MGJJ-MG2-051.hdf5
Measurement: IcRn
  - JS311-BHENL001-2JJ-2HB-5MGJJ-MG2-051.hdf5
    - level 0: {'sample': Classifier(column_name=None, values=('2JJ-2HB-5MGJJ-MG2',), requires_filtering=False)}
    - level 1: {'GV': Classifier(column_name='SM2 - Source voltage', values=(-7.0, -6.5, -6.0, -5.5, -5.0, -4.5, -4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0), requires_filtering=True)}
- Group IcRn:
  {'files': array(['/Users/bh/Labber/Data/2020/12/Data_1202/JS311-BHENL001-2JJ-2HB-5MGJJ-MG2-051.hdf5'],
      dtype=object)}
  - Group sample::2JJ-2HB-5MGJJ-MG2:
    {'sample': '2JJ-2HB-5MGJJ-MG2'}
    - Group GV::-0.5:
      {'GV': -0.5}
      - Dataset Bias: shape (141, 201), dtype float64, min -3e-06, max -3e-06
      - Dataset Magnet Source - Source current: shape (141, 201), dtype float64, min -0.07, max -0.07
      - Dataset Voltage Drop: shape (141, 201), dtype float64, min -0.007584390230476856, max -0.007584390230476856
    - Group GV::